<a href="https://colab.research.google.com/github/oldairjsilva/IGTI/blob/main/Projeto_Aplicado_CD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Projeto Aplicado

O projeto aplicado tem como desafio coletar os dados financeiros de empresa listada na Bolsa de Valores e comparar com o Valor Intríseco calculado pela fórmula de Graham. 
O objetivo seria verificar o histórico da margem de segurança entre a diferença entre a cotação diária e valor intrínseco calculado pela fórmula de Graham.

#### Base de dados CVM
#### http://dados.cvm.gov.br

#### Configurações iniciais

In [13]:
!pip install wget


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=c23823e94181b40b1b4ddd3ee9bf091e2cda281a2bb2bcef84836aca3e3239fe
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [11]:
import pandas as pd
import wget
from zipfile import ZipFile
from google.colab.data_table import DataTable
import plotly.graph_objects as go
import numpy as np
from google.colab import drive
import os
from os import path

#### 1. Coletar dados financeiros das empresas na CVM
Acessar dados financeiros de empresas listadas da Bolsa de Valores na CVM e criar arquivos históricos

In [12]:
# Montar drive para armazenar arquivos
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Coletar dados financeiros das empresas na CVM
# Função para baixar arquivos CVM
def baixarArquivosCVM(AnoInicio, AnoFim):
  url_base = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'
  arquivos_zip = []
  # Criar pasta DADOS no google drive
  if path.exists('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS') == False:
    os.mkdir('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS')

  os.chdir('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS')

  # Cria as descrições dos arquivos pelos anos informados
  for ano in range(AnoInicio,AnoFim):
    arquivos_zip.append(f'dfp_cia_aberta_{ano}.zip')
  arquivos_zip

  # 1. Baixar arquivos .zip da CVM considerando os anos de início e fim informados
  
  # Faz o download dos arquivos da CVM
  for arq in arquivos_zip:
    wget.download(url_base+arq)

  # 2. Descompactar os arquivos na pasta CVM
  for arq in arquivos_zip:
    ZipFile(arq, 'r').extractall('CVM')

  # 3. Unificar os arquivos por seus tipos. Ex. BPP/DRE
  
  # Cria o diretorio para unificar os dados
  if path.exists('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS/UNIFICADOS') == False:
    os.mkdir('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS/UNIFICADOS')  

  nomes = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DFC_MD_con', 'DFC_MD_ind', 'DFC_MI_con', 'DFC_MI_ind', 'DMPL_con', 'DMPL_ind', 'DRA_con', 'DRA_ind', 'DRE_con', 'DRE_ind', 'DVA_con', 'DVA_ind']
  for nome in nomes:
    arquivo = pd.DataFrame()

    for ano in range(AnoInicio,AnoFim):
      arquivo = pd.concat([arquivo, pd.read_csv(f'CVM/dfp_cia_aberta_{nome}_{ano}.csv', sep=';', decimal=',', encoding='ISO-8859-1')])
    arquivo.to_csv(f'UNIFICADOS/dfp_cia_aberta_{nome}_{AnoInicio}-{AnoFim}.csv', index = False)


In [ ]:
# Executar a função para baixar arquivos CVM
baixarArquivosCVM(2010, 2023) 

#### 2. Acessar dados financeiros para gerar indicadores fundamentalistas e calcular a fórmula de Graham


In [8]:
# Analisar DRE e BPP para calcular indicadores/fórmula de Graham de empresas selecionadas: BCO BRASIL S.A., ENGIE BRASIL ENERGIA S.A. e WEG S.A.
# Função para gerar indicadores fundamentalistas
def gerarIndicadoresFundamentalistas(Empresa):
  # Acesso os arquivos da CVM - DRE e BPP
  dre = pd.read_csv('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS/UNIFICADOS/dfp_cia_aberta_DRE_con_2010-2023.csv')
  bpp = pd.read_csv('/content/gdrive/MyDrive/IGTI/ProjetoAplicado/DADOS/UNIFICADOS/dfp_cia_aberta_BPP_con_2010-2023.csv')

  # Seleciona o último exercício informado
  dre = dre[dre['ORDEM_EXERC'] == "ÚLTIMO"]
  bpp = bpp[bpp['ORDEM_EXERC'] == "ÚLTIMO"]

  # Seleciona a empresa a partir do parametro passado
  empresaDRE = dre[dre['DENOM_CIA'].isin([Empresa])]
  empresaBPP = bpp[bpp['DENOM_CIA'].isin([Empresa])]

  # Seleciona as informações de LPA(lucro por ação), Lucro e Patrimonio Liquido a partir das contas contabeis
  contaLPA = empresaDRE[empresaDRE['CD_CONTA'] == '3.99.01.01']
  contaLucro = empresaDRE[empresaDRE['DS_CONTA'].isin(['Lucro/Prejuízo Consolidado do Período', 'Lucro ou Prejuízo Líquido Consolidado do Período'])]
  contaPatrimonioLiquido = empresaBPP[empresaBPP['DS_CONTA'] == 'Patrimônio Líquido Consolidado']

  # Seleciona as informações e renomeia a coluna - LPA
  contaLPA = contaLPA[['DT_REFER', 'DENOM_CIA', 'VL_CONTA']]
  contaLPA.rename({'VL_CONTA':'LPA'}, axis=1, inplace=True)
  contaLPA['LPA'] = round(contaLPA['LPA'],2)

  # Seleciona as informações e renomeia a coluna - Lucro
  contaLucro = contaLucro[['DT_REFER', 'DENOM_CIA', 'VL_CONTA']]
  contaLucro.rename({'VL_CONTA':'LUCRO'}, axis=1, inplace=True)
  contaLucro['LUCRO'] = contaLucro['LUCRO']*1000

  # Seleciona as informações e renomeia a coluna - Patrimonio Liquido
  contaPatrimonioLiquido = contaPatrimonioLiquido[['DT_REFER', 'DENOM_CIA', 'VL_CONTA']]
  contaPatrimonioLiquido.rename({'VL_CONTA':'PATRIMONIO_LIQUIDO'}, axis=1, inplace=True)
  contaPatrimonioLiquido['PATRIMONIO_LIQUIDO'] = contaPatrimonioLiquido['PATRIMONIO_LIQUIDO']*1000

  # Cria um data frame unificado de contaLPA, contaLucro e contaPatrimonioLiquido
  contaLPA_contaLucro = pd.merge(contaLPA, contaLucro, on = ['DENOM_CIA', 'DT_REFER']).sort_values('DT_REFER')
  contaLPA_contaLucro_contaPatrimonioLiquido = pd.merge(contaLPA_contaLucro, contaPatrimonioLiquido, on = ['DENOM_CIA', 'DT_REFER']).sort_values('DT_REFER')

  # Calcula a quantidade de ações a partir do lucro dividido por lucro por ação
  contaLPA_contaLucro_contaPatrimonioLiquido['QTDE_ACOES'] = contaLPA_contaLucro_contaPatrimonioLiquido['LUCRO']/contaLPA_contaLucro_contaPatrimonioLiquido['LPA']
  contaLPA_contaLucro_contaPatrimonioLiquido['QTDE_ACOES'] = contaLPA_contaLucro_contaPatrimonioLiquido['QTDE_ACOES'].values.astype(np.int64)

  # Calcula o VPA(valor patrimonial por ação) dividindo o valor do patrimonio liquido pela quantidade de ações
  contaLPA_contaLucro_contaPatrimonioLiquido['VPA'] = round(contaLPA_contaLucro_contaPatrimonioLiquido['PATRIMONIO_LIQUIDO']/contaLPA_contaLucro_contaPatrimonioLiquido['QTDE_ACOES'],2)

  # Calcula o valor intrinseco da Formula de Graham
  contaLPA_contaLucro_contaPatrimonioLiquido['VI_GRAHAM'] = round(np.sqrt(contaLPA_contaLucro_contaPatrimonioLiquido['VPA'] * contaLPA_contaLucro_contaPatrimonioLiquido['LPA'] * 22.5),2)

  # Cria o data frame indicadoresFundamentalistas
  indicadoresFundamentalistas = contaLPA_contaLucro_contaPatrimonioLiquido

  # Cria o index a partir da DT_REFER
  indicadoresFundamentalistas.index = pd.to_datetime(indicadoresFundamentalistas['DT_REFER'])

  # Remove a coluna DT_REFER depois de criar o indice
  del indicadoresFundamentalistas['DT_REFER']

  # Retorna o data frame com as informações tratadas
  return indicadoresFundamentalistas

In [ ]:
DataTable(gerarIndicadoresFundamentalistas('WEG S.A.'))

In [ ]:
DataTable(gerarIndicadoresFundamentalistas('BCO BRASIL S.A.'))

In [ ]:
DataTable(gerarIndicadoresFundamentalistas('ENGIE BRASIL ENERGIA S.A.'))

In [ ]:
#empresas = dre[['DENOM_CIA', 'CD_CVM']].drop_duplicates().set_index('CD_CVM')
#DataTable(empresas)

In [ ]:
#DataTable(empresaDRE[['CD_CONTA', 'DS_CONTA']].drop_duplicates().set_index('CD_CONTA'))
#DataTable(empresaBPP[['CD_CONTA', 'DS_CONTA']].drop_duplicates().set_index('CD_CONTA'))

#### 3. Baixar cotações diárias através do Yahoo Finance

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import yfinance as yf

In [ ]:
pricesBBAS3 = yf.download('BBAS3.SA', start='2011-01-01')[['Close', 'Adj Close']]
pricesWEGE3 = yf.download('WEGE3.SA', start='2011-01-01')[['Close', 'Adj Close']]
pricesEGIE3 = yf.download('EGIE3.SA', start='2011-01-01')[['Close', 'Adj Close']]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
pricesEGIE3.head()

,Close,Adj Close
Date,,
2011-01-03,21.440001,10.339413
2011-01-04,21.160000,10.204383
2011-01-05,21.520000,10.377994
2011-01-06,21.799999,10.513021
2011-01-07,21.879999,10.551603


#### 4. Juntar as cotações diárias com os indicadores fundamentalistas

In [ ]:
indicadoresBBAS3 = pricesBBAS3.join(gerarIndicadoresFundamentalistas('BCO BRASIL S.A.'), how = 'outer')
indicadoresWEGE3 = pricesWEGE3.join(gerarIndicadoresFundamentalistas("WEG S.A."), how = 'outer')
indicadoresEGIE3 = pricesEGIE3.join(gerarIndicadoresFundamentalistas("ENGIE BRASIL ENERGIA S.A."), how = 'outer')

In [ ]:
indicadoresEGIE3.head()

In [ ]:
# Preencher os dados fundamentalistas a partir da data de disponibilização para o ano seguinte
indicadoresBBAS3.fillna(method='ffill', inplace = True)
indicadoresWEGE3.fillna(method='ffill', inplace = True)
indicadoresEGIE3.fillna(method='ffill', inplace = True)

In [ ]:
indicadoresEGIE3

In [ ]:
# Remover as linhas nulas referente aos dados de Fechamento(Close) e Fechamento ajustado(Adj Close)
indicadoresBBAS3.dropna(inplace = True)
indicadoresWEGE3.dropna(inplace = True)
indicadoresEGIE3.dropna(inplace = True)

In [ ]:
indicadoresEGIE3

In [56]:
# Coletar cotações das ações no Yahoo Finance das empresas selecionadas
# Função para gerar Cotações com os indicadores fundamentalistas das empresas selecionadas
def gerarCotacoesComIndicadores(Ativo, Empresa):
  # Coletar cotações das ações no Yahoo Finance das empresas selecionadas
  cotacoes = yf.download(Ativo, start='2011-01-01')[['Close', 'Adj Close']]

  # Unificar os Dados financeiros/indicadores/fórmula de Graham com as cotações
  indicadores = cotacoes.join(gerarIndicadoresFundamentalistas(Empresa), how = 'outer')

  # Preencher os indicadores/dados financeiros a partir da data de disponibilização para o ano seguinte
  indicadores.fillna(method='ffill', inplace = True)

  # Remover as linhas nulas referente aos dados de Fechamento(Close) e Fechamento ajustado(Adj Close)
  indicadores.dropna(inplace = True)

  # Retorna o data frame com as informações tratadas
  return indicadores

In [ ]:
indicadoresBBAS3 = gerarCotacoesComIndicadores('BBAS3.SA', 'BCO BRASIL S.A.')
indicadoresWEGE3 = gerarCotacoesComIndicadores('WEGE3.SA', 'WEG S.A.')
indicadoresEGIE3 = gerarCotacoesComIndicadores('EGIE3.SA', 'ENGIE BRASIL ENERGIA S.A.')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
indicadores = pd.concat([indicadoresBBAS3, indicadoresWEGE3, indicadoresEGIE3])

In [ ]:
indicadores

In [4]:
listaAtivosEmpresas = [['BBAS3.SA', 'BCO BRASIL S.A.'], ['WEGE3.SA', 'WEG S.A.'], ['EGIE3.SA', 'ENGIE BRASIL ENERGIA S.A.']]

In [40]:
for ativoEmpresa in listaAtivosEmpresas:
  print(ativoEmpresa[1])

BCO BRASIL S.A.
WEG S.A.
ENGIE BRASIL ENERGIA S.A.


In [55]:
# Criar a visualização
# Função para gerar data frame unificado com as empresas selecionadas - Cotações x indicadores fundamentalistas
# Objetivo: Utilizar na criação do dashboard
def criarDataFrameParaDashboard(ListaAtivosEmpresas):
  # Cria data frame vazio
  indicadoresUnificados = pd.DataFrame()

  # Gerar cotações com indicadores de cada empresa e fazer a unificação de todas em um mesmo data frame para ser utilizado na criação do dashboard
  for ativoEmpresa in listaAtivosEmpresas:
    indicadoresEmpresa = gerarCotacoesComIndicadores(ativoEmpresa[0], ativoEmpresa[1])
    # Unifica os data frames das empresas
    indicadoresEmpresasUnificados = pd.concat([indicadoresUnificados, indicadoresEmpresa])
  
  return indicadoresEmpresasUnificados

In [52]:
dataFrameDashboard = criarDataFrameParaDashboard(listaAtivosEmpresas)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [53]:
dataFrameDashboard

,Close,Adj Close,DENOM_CIA,LPA,LUCRO,PATRIMONIO_LIQUIDO,QTDE_ACOES,VPA,VI_GRAHAM
2011-01-03,31.400000,14.923953,BCO BRASIL S.A.,4.17,1.133034e+10,5.441894e+10,2.717109e+09,20.03,43.35
2011-01-04,31.530001,14.985735,BCO BRASIL S.A.,4.17,1.133034e+10,5.441894e+10,2.717109e+09,20.03,43.35
2011-01-05,31.980000,15.199616,BCO BRASIL S.A.,4.17,1.133034e+10,5.441894e+10,2.717109e+09,20.03,43.35
2011-01-06,31.700001,15.066534,BCO BRASIL S.A.,4.17,1.133034e+10,5.441894e+10,2.717109e+09,20.03,43.35
2011-01-07,30.950001,14.710078,BCO BRASIL S.A.,4.17,1.133034e+10,5.441894e+10,2.717109e+09,20.03,43.35
...,...,...,...,...,...,...,...,...,...
2022-06-27,41.200001,41.200001,ENGIE BRASIL ENERGIA S.A.,1.92,1.564970e+09,7.932597e+09,8.150885e+08,9.73,20.50
2022-06-28,41.490002,41.490002,ENGIE BRASIL ENERGIA S.A.,1.92,1.564970e+09,7.932597e+09,8.150885e+08,9.73,20.50
2022-06-29,41.630001,41.630001,ENGIE BRASIL ENERGIA S.A.,1.92,1.564970e+09,7.932597e+09,8.150885e+08,9.73,20.50
2022-06-30,41.299999,41.299999,ENGIE BRASIL ENERGIA S.A.,1.92,1.564970e+09,7.932597e+09,8.150885e+08,9.73,20.50


In [ ]:
# Exportar os dados em formato csv para fazer dashboard
dataFrameDashboard.to_csv('dataFrameDashboard.csv', index = True, sep = ';', decimal = '.')

In [ ]:
# Exportar os dados em formato csv para fazer dashboard
dataFrameDashboard.to_excel("dataFrameDashboard.xlsx",sheet_name='dataFrameDashboard')  

In [ ]:
# Gráfico comparando o Valor Intríseco de Graham com o Valor da Cotação diária
fig = go.Figure()
fig.add_trace(go.Scatter(x = indicadoresBBAS3.index, y = indicadoresBBAS3['VI_GRAHAM'],  name='Valor Intrínseco - GRAHAM'))
fig.add_trace(go.Scatter(x = indicadoresBBAS3.index, y = indicadoresBBAS3['Close'],  name='BBAS3'))
fig.add_trace(go.Scatter(x = indicadoresBBAS3.index, y = indicadoresBBAS3['Adj Close'],  name='BBAS3_Ajustado'))

In [ ]:
# Gráfico comparando o Valor Intríseco de Graham com o Valor da Cotação diária
fig = go.Figure()
fig.add_trace(go.Scatter(x = indicadoresEGIE3.index, y = indicadoresEGIE3['VI_GRAHAM'],  name='Valor Intrínseco - GRAHAM'))
fig.add_trace(go.Scatter(x = indicadoresEGIE3.index, y = indicadoresEGIE3['Close'],  name='EGIE3'))
fig.add_trace(go.Scatter(x = indicadoresEGIE3.index, y = indicadoresEGIE3['Adj Close'],  name='EGIE3_Ajustado'))

In [ ]:
# Gráfico comparando o Valor Intríseco de Graham com o Valor da Cotação diária
fig = go.Figure()
fig.add_trace(go.Scatter(x = indicadoresWEGE3.index, y = indicadoresWEGE3['VI_GRAHAM'],  name='Valor Intrínseco - GRAHAM'))
fig.add_trace(go.Scatter(x = indicadoresWEGE3.index, y = indicadoresWEGE3['Close'],  name='WEGE3'))
fig.add_trace(go.Scatter(x = indicadoresWEGE3.index, y = indicadoresWEGE3['Adj Close'],  name='WEGE3_Ajustado'))